In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Hypothesis Test - 1

## Completion Rate - Z-test

- Null Hypothesis (H0) - The difference between the completion rate of the two versions of the online process is statistically not significant
- Alternative Hypothesis (Ha)- The difference between the completion rate of the two versions of the online process is statistically significant

In [5]:
df = pd.read_csv("../Data/Processed/df_kpi_clients.csv")

# Compute totals + completed per group
summary = (
    df.groupby("variation")["completion_status"]
      .agg(total="count", completed="sum")   # completion_status is 1/0
      .reset_index()
)

# Completion rate (%)
summary["completion_rate_pct"] = 100 * summary["completed"] / summary["total"]

print(summary)

# 3) Extract values for hypothesis testing
c_total = int(summary.loc[summary["variation"] == "Control", "total"].iloc[0])
c_completed = int(summary.loc[summary["variation"] == "Control", "completed"].iloc[0])

t_total = int(summary.loc[summary["variation"] == "Test", "total"].iloc[0])
t_completed = int(summary.loc[summary["variation"] == "Test", "completed"].iloc[0])


  variation  total  completed  completion_rate_pct
0   Control  23532      15434            65.587285
1      Test  26968      18687            69.293236


In [11]:
p = 0.05

# completion rates
p_c = c_completed / c_total
p_t = t_completed / t_total

# pooled proportion
p_pool = (c_completed + t_completed) / (c_total + t_total)

# standard error - how much random fluctuation in the completion-rate difference is expected just by chance
se = np.sqrt(p_pool * (1 - p_pool) * (1/c_total + 1/t_total))

# Z statistic
z = (p_t - p_c) / se

# one-sided p-value
p_value = 1 - stats.norm.cdf(z)

print(f"Control completion rate: {p_c:.4f}")
print(f"Test completion rate: {p_t:.4f}")
print(f"Z-statistic: {z:.4f}")
print(f"P-value: {p_value:.2e}")


if p_value < p:
    print("Reject H0: The difference between the completion rate of the two versions of the online process is statistically significant")
else:
    print("Fail to reject H0.")


Control completion rate: 0.6559
Test completion rate: 0.6929
Z-statistic: 8.8745
P-value: 0.00e+00
Reject H0: The difference between the completion rate of the two versions of the online process is statistically significant


# Hypothesis Test - 2

## Completion Rate with a Cost-Effectiveness Threshold - one sided Z-test

- Null Hypothesis (H0) - The increase in completion rate for the Test group compared to the Control group is **less than or equal** to 5 percentage points
- Alternative Hypothesis (Ha)- The increase in completion rate for the Test group compared to the Control group is **greater than** to 5 percentage points

In [13]:
c_completed, c_total = 15434, 23532
t_completed, t_total = 18687, 26968

p = 0.05   # 5 percentage-points (absolute lift)

# proportions
p_c = c_completed / c_total
p_t = t_completed / t_total
diff = p_t - p_c

# standard error for difference
se = np.sqrt(p_t * (1 - p_t) / t_total + p_c * (1 - p_c) / c_total)

# z statistic testing diff > delta0
z = (diff - p) / se

# one-sided p-value
p_value = 1 - stats.norm.cdf(z)

print(f"Observed lift (Test-Control): {diff:.4f}")
print(f"z = {z:.4f}, p-value = {p_value:.6f}")

if p_value < p:
    print("Reject H0: The increase in completion rate for the Test group compared to the Control group is greater than to 5 percentage points")
else:
    print("Fail to reject H0: The increase in completion rate for the Test group compared to the Control group is less than or equal to 5 percentage points")


Observed lift (Test-Control): 0.0371
z = -3.0950, p-value = 0.999016
Fail to reject H0: The increase in completion rate for the Test group compared to the Control group is less than or equal to 5 percentage points
